In [0]:
dbutils.library.restartPython()

In [0]:

%sh

pip install azure-identity
pip install azure-monitor-query

  Obtaining dependency information for azure-identity from https://files.pythonhosted.org/packages/3d/9f/1f9f3ef4f49729ee207a712a5971a9ca747f2ca47d9cbf13cf6953e3478a/azure_identity-1.21.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 646.6 kB/s eta 0:00:00
  Obtaining dependency information for azure-core>=1.31.0 from https://files.pythonhosted.org/packages/07/b7/76b7e144aa53bd206bf1ce34fa75350472c3f69bf30e5c8c18bc9881035d/azure_core-1.33.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.2 MB/s eta 0:00:00
  Obtaining dependency information for msal>=1.30.0 from https://files.pythonhosted.org/packages/04/bf/81516b9aac7fd867709984d08eb4db1d2e3fe1df795c8e442cde9b568962/msal-1.32.3-py3-none-any.whl.metadata
  Obtaining dependency information for msal-extensions>=1.2.0 from https://files.pythonhosted.org/packages/5e/75/bd9b7bb966668920f06b200e84454c8f3566b102183bc55c5473d96cb2b9/msal_extensions-1.3.1-py3-none-any.


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


  Obtaining dependency information for azure-monitor-query from https://files.pythonhosted.org/packages/cb/71/bdf03aef699d9a17d80bcaa8255c20e596ed591daa8a42a21854ddc4277d/azure_monitor_query-1.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 1.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [0]:
vnet_logs = '''

NTANetAnalytics
| where FlowType != '' and SubType == 'FlowLog'
| extend SrcPIP = substring(SrcPublicIps, 0, indexof(SrcPublicIps, "|"))
| extend DestPIP = substring(DestPublicIps, 0, indexof(DestPublicIps, "|"))

//check "location" of src node
| extend SrcNodeInOrOutVNet = iif(SrcIp != '' and FlowType != 'MaliciousFlow', 'NodeInsideVNet',
                            iif(FlowType == 'AzurePublic' and SrcPublicIps != '', 'NodeOutsideVNet_PaaS', 
                                iif(FlowType == 'ExternalPublic' and SrcPublicIps != '', 'NodeOutsideAzure',
                                //adversary from outside of Azure 
                                iif (FlowType =='MaliciousFlow' and FlowDirection == 'Inbound' and SrcIp != '', 'NodeMalicious',
                                    //adversary from inside of Azure 
                                    iif(FlowType =='MaliciousFlow' and FlowDirection == 'Outbound', 'NodeInsideVNet', 'NodeUnknown')
                                )
                              )
                            )
                          )
| extend SrcSubnetSplitted =  split(SrcSubnet, '/')
| extend SrcRG = tostring(SrcSubnetSplitted[0])
| extend SrcVNet = tostring(SrcSubnetSplitted[1])
| extend SrcSubnetName = tostring(SrcSubnetSplitted[2])

//check "location" of dest node
| extend DestNodeInOrOutVNet = iif(DestIp != '' and FlowType != 'MaliciousFlow', 'NodeInsideVNet',
                            iif(FlowType in ('AzurePublic') and DestPublicIps != '', 'NodeOutsideVNet_PaaS', 
                                iif(FlowType == 'ExternalPublic' and DestPublicIps != '', 'NodeOutsideAzure',
                                    //adversary from outside of Azure 
                                    iif(FlowType == 'MaliciousFlow' and FlowDirection== 'Inbound' and DestIp != '', 'NodeInsideVNet',
                                        //adversary from inside of Azure 
                                        iif(FlowType == 'MaliciousFlow' and FlowDirection== 'Outbound', 'NodeMalicious', 'NodeUnknown')
                                )
                              )
                            )
                          )
| extend DestSubnetSplitted =  split(DestSubnet, '/')
| extend DestRG = tostring(DestSubnetSplitted[0])
| extend DestVNet = tostring(DestSubnetSplitted[1])
| extend DestSubnetName = tostring(DestSubnetSplitted[2])

| join kind=leftouter 
(NTAIpDetails 
| project ['SrcPIP_Ip']=Ip, ['SrcPIP_PublicIpDetails']=PublicIpDetails, ['SrcPIP_ThreatType']=ThreatType, ['SrcPIP_Location']=Location, ['SrcPIP_Url']=Url, Src_NTAIpDetails=1
)
on $left.SrcPIP == $right.SrcPIP_Ip

| join kind=leftouter
    (
        NTAIpDetails 
        | project ['DestPIP_Ip']=Ip, ['DestPIP_PublicIpDetails']=PublicIpDetails, ['DestPIP_ThreatType']=ThreatType, ['DestPIP_Location']=Location, ['DestPIP_Url']=Url, Dest_NTAIpDetails=1
    )
on $left.DestPIP == $right.DestPIP_Ip

// set SrcPIP_Ip if exist
| extend SrcIp = iif(SrcPIP_Ip != '', SrcPIP_Ip, SrcIp)

| extend SrcName = iif(SrcApplicationGateway != '', SrcApplicationGateway, 
                     iif(SrcLoadBalancer != '', SrcLoadBalancer, 
                        iif(SrcPIP_PublicIpDetails != '', SrcPIP_PublicIpDetails, 
                          iif(SrcSubnetName == 'azurebastionsubnet', 'bastion-vm', 
                            iif(SrcVm != '', SrcVm, '' )))))

| extend SrcName = iif(indexof(SrcName, '/',0) > 0, split(SrcName, '/')[-1], SrcName)

// set DestPIP_Ip if exist
| extend DestIp = strcat(iif(DestPIP_Ip != '', DestPIP_Ip, DestIp), ':', DestPort)

| extend DestName = iif(DestApplicationGateway != '', DestApplicationGateway, 
                        iif(DestLoadBalancer != '', DestLoadBalancer,
                            iif(DestPIP_PublicIpDetails != '', DestPIP_PublicIpDetails,
                               iif(DestSubnetName == 'azurebastionsubnet', 'bastion-vm', 
                                  iif(DestVm != '',  DestVm, '' )))))


| extend DestName = iif(indexof(DestName, '/',0) > 0, split(DestName, '/')[-1], DestName)


| extend protocol = tolower(strcat(L4Protocol,  iif(L7Protocol != 'Unknown', strcat(':', L7Protocol), ''))) 

| summarize BytesSrcToDest = max(BytesSrcToDest), BytesDestToSrc = max(BytesDestToSrc)
            by Src_NTAIpDetails, Dest_NTAIpDetails, FlowType, FlowDirection, ConnectionType, protocol,
                    SrcNodeInOrOutVNet, SrcSubscription, SrcRG, SrcVNet, SrcSubnetName , SrcIp, SrcName,
                    DestNodeInOrOutVNet, DestSubscription, DestRG, DestVNet, DestSubnetName, DestName,
                    SrcPIP_ThreatType, SrcPIP_Location, SrcPIP_Url,
                    DestIp, DestPIP_ThreatType, DestPIP_Location, DestPIP_Url


| extend SrcToDestDataSize = format_bytes(BytesSrcToDest, 2)
| extend DestToSrcDataSize = format_bytes(BytesDestToSrc, 2)  



//SrcName and DestName can be 
    // VM
    // App Gateway
    // Load Balancer
    // Private Endpoint
    // PaaS services
    // external endpoints which has no name and is empty
    // on-premise which has no name and is empty
| distinct Src_NTAIpDetails, Dest_NTAIpDetails, FlowType, FlowDirection, ConnectionType, protocol,
        SrcNodeInOrOutVNet, SrcSubscription, SrcRG, SrcVNet, SrcSubnetName , SrcIp, SrcName, SrcToDestDataSize,
        DestNodeInOrOutVNet, DestSubscription, DestRG, DestVNet, DestSubnetName, DestName, DestToSrcDataSize,
        SrcPIP_ThreatType, SrcPIP_Location, SrcPIP_Url,
        DestIp, DestPIP_ThreatType, DestPIP_Location, DestPIP_Url


// SrcPIP_PublicIpDetails , DestPIP_PublicIpDetails

'''

In [0]:
from azure.monitor.query import LogsQueryClient
from azure.identity import DefaultAzureCredential
import datetime
from datetime import timedelta
import pandas as pd
import math
import json

credential = DefaultAzureCredential()
client = LogsQueryClient(credential)

now = datetime.datetime.now()
mins_ago = now - timedelta(minutes=30)

response = client.query_workspace(
            workspace_id= 'c596239e-beda-4999-a79f-58e34d9881e4',
            query=vnet_logs,
            timespan=timedelta(hours=2)
        )

table = response.tables[0]

df = pd.DataFrame(data=table.rows, columns=table.columns)


In [0]:
print(df['FlowType'].unique())

['AzurePublic' 'ExternalPublic' 'InterVNet' 'IntraVNet' 'UnknownPrivate']


In [0]:

# drop all logs where they are: Inbound-ExternalPublic and found in NTAIpDetails table
externalpublic_inbound_NTAIpDetails_match_indexes = df[(df['FlowDirection']=='Inbound') & (df['FlowType']=='ExternalPublic') & (df['SrcName'] != '')].index

tempdf = df.drop(externalpublic_inbound_NTAIpDetails_match_indexes)

# update SrcName by Subnet
                                               
tempdf['SrcName'] = tempdf.apply(lambda x: x['SrcSubnetName'] if x['SrcName']=='' else x['SrcName'], axis=1)

maindf = tempdf.drop_duplicates(subset=['SrcName', 'DestName'])

maindf.fillna('', inplace=True)

#maindf = maindf[maindf['FlowType'] == 'InterVNet']

print(len(maindf.index))

maindf





92


/root/.ipykernel/875/command-5477959247676144-3026362402:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maindf.fillna('', inplace=True)


,Src_NTAIpDetails,Dest_NTAIpDetails,FlowType,FlowDirection,ConnectionType,protocol,SrcNodeInOrOutVNet,SrcSubscription,SrcRG,SrcVNet,SrcSubnetName,SrcIp,SrcName,SrcToDestDataSize,DestNodeInOrOutVNet,DestSubscription,DestRG,DestVNet,DestSubnetName,DestName,DestToSrcDataSize,SrcPIP_ThreatType,SrcPIP_Location,SrcPIP_Url,DestIp,DestPIP_ThreatType,DestPIP_Location,DestPIP_Url
0,,1.0,AzurePublic,Outbound,,tcp:https,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.5,vnetgw-cwhd-onpremise-VM-192.166.0.5,357.88 KB,NodeOutsideVNet_PaaS,,,,,OneDsCollector,657.19 KB,,,,104.208.16.91:443,,centralus,
1,,1.0,AzurePublic,Outbound,,tcp:https,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-ingress,apim,172.16.2.5,unknown-vm,30.63 MB,NodeOutsideVNet_PaaS,,,,,AzureKeyVault,33.27 MB,,,,13.67.8.104:443,,southeastasia,
2,,1.0,AzurePublic,Outbound,,tcp:https,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.4,vnetgw-cwhd-onpremise-VM-192.166.0.4,2.93 KB,NodeOutsideVNet_PaaS,,,,,OneDsCollector,5.76 KB,,,,13.69.239.78:443,,northeurope,
4,,1.0,AzurePublic,Outbound,,tcp:https,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,vm,172.18.1.4,vm-cwhd-s2s-azure,13.6 KB,NodeOutsideVNet_PaaS,,,,,OneDsCollector,11.89 KB,,,,13.69.239.78:443,,northeurope,
5,,1.0,AzurePublic,Outbound,,tcp:entextxid,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,172.18.0.5,vpngw-cwhd-s2s-azure-VM-172.18.0.5,6.26 KB,NodeOutsideVNet_PaaS,,,,,AzureMonitor,850 Bytes,,,,13.86.223.176:12000,,westus,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,1.0,,AzurePublic,Inbound,,tcp,NodeOutsideVNet_PaaS,,,,,40.80.36.0,AzureCloud,128 Bytes,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-ingress,apim,unknown-vm,0 Bytes,,koreacentral,,172.16.2.5:21464,,,
335,1.0,,AzurePublic,Inbound,,tcp,NodeOutsideVNet_PaaS,,,,,40.80.36.0,AzureCloud,128 Bytes,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-management,vms,vm-build-agent-win-server-1,0 Bytes,,koreacentral,,10.0.1.75:21620,,,
1199,1.0,,AzurePublic,Inbound,,tcp:pcsync-https,NodeOutsideVNet_PaaS,,,,,20.195.86.100,GatewayManager,27.33 KB,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,vnetgw-cwhd-onpremise-VM-192.166.0.4,26.88 KB,,southeastasia,,192.166.0.4:8443,,,
1200,1.0,,AzurePublic,Inbound,,tcp:pcsync-http,NodeOutsideVNet_PaaS,,,,,20.195.86.100,GatewayManager,28.47 KB,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,vpngw-cwhd-s2s-azure-VM-172.18.0.5,26.71 KB,,southeastasia,,172.18.0.5:8444,,,


In [0]:
    # {
    #   name: 'CNF',
    #   id: 'CNF',
    #   category: 1,
    #   ip: '12.20.11.23',
    #   subnet: "some text",
    #   vnet: '123'
    # },



# create nodes csv for IntraVNet
#tempdf = maindf[maindf['FlowType'] == 'IntraVNet']

src_nodes_final_df = pd.DataFrame()
src_nodes_final_df['id'] = maindf['SrcIp']
src_nodes_final_df['name'] = maindf['SrcName']
src_nodes_final_df['category'] = maindf['SrcNodeInOrOutVNet']
src_nodes_final_df['ip'] = maindf['SrcIp']
src_nodes_final_df['subnet'] = maindf['SrcSubnetName']
src_nodes_final_df['vnet'] = maindf['SrcVNet']


dest_nodes_final_df = pd.DataFrame()
dest_nodes_final_df['id'] = maindf['DestIp']
dest_nodes_final_df['name'] = maindf['DestName']
dest_nodes_final_df['category'] = maindf['DestNodeInOrOutVNet']
dest_nodes_final_df['ip'] = maindf['DestIp']
dest_nodes_final_df['subnet'] = maindf['DestSubnetName']
dest_nodes_final_df['vnet'] = maindf['DestVNet']

# # #id,title,subtitle,mainstat,secondarystat,color
combined_list = [src_nodes_final_df, dest_nodes_final_df]

final_nodes_df = pd.concat(combined_list)

final_nodes_df = final_nodes_df.drop_duplicates(subset=['id'])

final_nodes_json_str =  final_nodes_df.to_json(orient='records')

final_nodes_json_obj = json.loads(final_nodes_json_str)



print(final_nodes_json_str)


[{"id":"192.166.0.5","name":"vnetgw-cwhd-onpremise-VM-192.166.0.5","category":"NodeInsideVNet","ip":"192.166.0.5","subnet":"gatewaysubnet","vnet":"vnet-cwh-s2s-onpremise"},{"id":"172.16.2.5","name":"unknown-vm","category":"NodeInsideVNet","ip":"172.16.2.5","subnet":"apim","vnet":"vnet-ingress"},{"id":"192.166.0.4","name":"vnetgw-cwhd-onpremise-VM-192.166.0.4","category":"NodeInsideVNet","ip":"192.166.0.4","subnet":"gatewaysubnet","vnet":"vnet-cwh-s2s-onpremise"},{"id":"172.18.1.4","name":"vm-cwhd-s2s-azure","category":"NodeInsideVNet","ip":"172.18.1.4","subnet":"vm","vnet":"vnet-cwhd-s2s-azure"},{"id":"172.18.0.5","name":"vpngw-cwhd-s2s-azure-VM-172.18.0.5","category":"NodeInsideVNet","ip":"172.18.0.5","subnet":"gatewaysubnet","vnet":"vnet-cwhd-s2s-azure"},{"id":"172.18.0.4","name":"vpngw-cwhd-s2s-azure-VM-172.18.0.4","category":"NodeInsideVNet","ip":"172.18.0.4","subnet":"gatewaysubnet","vnet":"vnet-cwhd-s2s-azure"},{"id":"10.0.1.75","name":"vm-build-agent-win-server-1","category":"No

In [0]:

    # {
    #   source: 'CNF',
    #   target: 'SCPC',
    #   src_to_dest_data_size: '10 KB',
    #   dest_to_srct_data_size: '22 KB',
    #   flowType: 'Azure Public PaaS',
    #   label: {
    #     show: true,
    #     formatter: (params) => params.data.src_to_dest_data_size + '-> <- ' + params.data.dest_to_srct_data_size + '\n' + params.data.flowType,
    #     fontSize: 10
    #   },
    #   lineStyle: {
    #     curveness: 0.1
    #   }
    # }

# create edge

edges_df = pd.DataFrame()
edges_df['source'] = tempdf['SrcIp']
edges_df['target'] = tempdf['DestIp']
edges_df['value'] = tempdf['SrcToDestDataSize'] + '-' + tempdf['DestToSrcDataSize']
# edges_df['src_to_dest_data_size'] = tempdf['SrcToDestDataSize']
# edges_df['dest_to_srct_data_size'] = tempdf['DestToSrcDataSize']
# edges_df['flowType'] = tempdf['FlowType']
# edges_df['protocol'] = tempdf['protocol']

edges_df = edges_df.drop_duplicates(subset=['source', 'target'])

final_edges_json_str =  edges_df.to_json(orient='records')

final_edges_json_obj = json.loads(final_edges_json_str)

print(final_edges_json_str)


[{"source":"192.166.0.5","target":"104.208.16.91:443","value":"357.88 KB-657.19 KB"},{"source":"172.16.2.5","target":"13.67.8.104:443","value":"30.63 MB-33.27 MB"},{"source":"192.166.0.4","target":"13.69.239.78:443","value":"2.93 KB-5.76 KB"},{"source":"192.166.0.5","target":"13.69.239.78:443","value":"28.13 KB-15.85 KB"},{"source":"172.18.1.4","target":"13.69.239.78:443","value":"13.6 KB-11.89 KB"},{"source":"172.18.0.5","target":"13.86.223.176:12000","value":"6.26 KB-850 Bytes"},{"source":"172.18.0.4","target":"20.195.82.188:443","value":"9.44 MB-20.8 MB"},{"source":"172.18.0.4","target":"20.209.110.8:443","value":"106.44 KB-144.34 KB"},{"source":"172.18.0.5","target":"20.209.110.8:443","value":"154.75 KB-221.15 KB"},{"source":"192.166.0.5","target":"20.60.105.43:443","value":"148.62 KB-204.15 KB"},{"source":"172.18.1.4","target":"52.168.112.66:443","value":"55.04 KB-62.14 KB"},{"source":"192.166.0.4","target":"20.150.81.36:443","value":"1.35 MB-447.49 KB"},{"source":"172.18.1.4","ta

In [0]:

# category

unique_src_node_location = maindf['SrcNodeInOrOutVNet'].unique()
unique_dest_node_location = maindf['DestNodeInOrOutVNet'].unique()

unique_categories = list(set(unique_src_node_location) | set(unique_dest_node_location))

result = pd.DataFrame()
result['name'] = unique_categories

# for x in unique_src_node_location + unique_dest_node_location:
#     result.add(x)

# result = list(result)
unique_categories =  result.to_json(orient='records')

print(unique_categories)

[{"name":"NodeOutsideVNet_PaaS"},{"name":"NodeInsideVNet"},{"name":"NodeOutsideAzure"}]


In [0]:
final_json = {
    "nodes": final_nodes_json_obj,
    "edges": final_edges_json_obj,
    "categories": unique_categories
}

json_str = json.dumps(final_json)

with open('nodes_edges.txt', "w") as f:
    f.write(json_str)